# Setting up a mock DB to store Songs, users, and playlists
![](https://github.com/spotify-song/DS/blob/master/api/data/DB_tables_illu.png?raw=true)
### Tables:
    - User:
        - id
        - display_name
    - Tokens:
        - id
        - access_token
        - token_type
        - expries_in
        - refresh_token
        - scope
        - expires_at
        - user
    - Tracks
        - id
        - danceability
        - energy
        - loudness
        - mode
        - speechiness
        - accousticness
        - instrumentalness
        - valence
        - liveness
        - tempo
        - duration_ms
        - time_signature
    - UserPlaylist
        - id
        - u_id
        - uri
        - tracks_id

## Setting up db classes for data import

In [1]:
from os import getenv
from dotenv import load_dotenv

import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, backref, exists

load_dotenv()

# connecting to DB
engine = create_engine(getenv('DATABASE_URL'))
Session = sessionmaker(
                autocommit=False,
                autoflush=False,
                bind=engine
                )

Base = declarative_base()

In [2]:
class User(Base):
    __tablename__ = 'user'
    
    id = Column(String, primary_key=True, index=True)
    display_name = Column(String, unique=True, nullable=False)
    
    # Backref allows you to update values in a different table
    # set the table name then the back arg will be used as the psudo
    # column that will fill the foreign key in the corresponding table
    # in the case of the value below, the 'token' value will be used to
    # fill the 'user_id' column in the 'Tokens' table
    # Ex: Tokens(all_token_column_names=all_token_values, user_token=User(object))
    token = relationship('Tokens', backref='user_token')
    user_playlist = relationship('UserPlaylist', backref='user_id')
    
    
    def __repr__(self):
        return "<User(display_name='%s')>" % (self.display_name)



class Tokens(Base):
    __tablename__ = 'tokens'
    
    id = Column(Integer, primary_key=True, index=True)
    access_token = Column(String, unique=True, nullable=False)
    token_type = Column(String, unique=False, nullable=False)
    expires_in = Column(Integer, unique=False, nullable=False)
    refresh_token = Column(String, unique=True, nullable=False)
    scope = Column(String, unique=False, nullable=False)
    expires_at = Column(Integer, unique=False, nullable=False)
    user = Column(String, ForeignKey('user.id'))
    
    def __repr__(self):
        return "<Tokens(\
        access_token='%s',\
        token_type='%s',\
        expires_in='%s',\
        refresh_token='%s',\
        scope='%s',\
        expires_at='%s',\
        user='%s')>" % (
            self.access_token,
            self.token_type,
            self.expires_in,
            self.refresh_token,
            self.scope,
            self.expires_at,
            self.user
            )


class Tracks(Base):
    __tablename__ = 'tracks'
    
    id = Column(String, primary_key=True)
    danceability = Column(Float)
    energy = Column(Float)
    key = Column(Integer)
    loudness = Column(Float)
    mode = Column(Integer)
    speechiness = Column(Float)
    acousticness = Column(Float)
    instrumentalness = Column(Float)
    valence = Column(Float)
    liveness = Column(Float)
    tempo = Column(Float)
    duration_ms = Column(Integer)
    time_signature = Column(Integer)
    
    user_ply_lst = relationship('UserPlaylist', backref='track_id', uselist=False)
    
    def __repr__(self):
        return "<Tracks Data(danceability='%s', energy='%s', key='%s', loudness='%s',\
                mode='%s', speechiness='%s', acousticness='%s', instrumentalness='%s',\
                liveness='%s', valence='%s', tempo='%s', duration_ms='%s',\
                time_signature='%s')" % (
                        self.danceability,
                        self.energy,
                        self.key,
                        self.loudness,
                        self.mode,
                        self.speechiness,
                        self.acousticness,
                        self.instrumentalness,
                        self.valence,
                        self.liveness,
                        self.tempo,
                        self.duration_ms,
                        self.time_signature
                        )


class UserPlaylist(Base):
    '''When running this class, make sure to set the user_id arg to the
        user object it corresponds to; same goes for the track_id arg
        (note: track_id is diff from tracks_id) to the tracks object
        user_id = User()object  to fill the u_id arg ForeignKey
        track_id = Tracks()object  to fill the tracks_id arg ForeignKey'''
    
    __tablename__ = 'user_playlist'
    
    id = Column(Integer, primary_key=True, index=True)
    u_id = Column(String, ForeignKey('user.id'))           # User_ID
    tracks_id = Column(String, ForeignKey('tracks.id'))    # Track_ID
    uri = Column(String)                                    # Playlist_URI
    
    def __repr__(self):
        return "<User Playlist(u_id='%s', tracks_id='%s', uri='%s')" % (
                                                    self.u_id,
                                                    self.tracks_id,
                                                    self.uri
                                                    )




## DB import flow needs to be:
    - User > tokens
    - Tracks > top_tracks
    - users/tracks > user_playlists

### Get the pertinent data using the UserData() class

In [3]:
import sys
sys.path.insert(0, '../')
from spotify_users import UserData
user = UserData()
user_dict = user.user_top_50(user_id='Agustinvargas')

In [4]:
# Token details
# user_dict['Tokens Info']['access_token']
# user_dict['Tokens Info']['token_type']
# user_dict['Tokens Info']['expires_in']
# user_dict['Tokens Info']['scope']
# user_dict['Tokens Info']['expires_at']
# user_dict['Tokens Info']['refresh_token']

# Data for a given user: Token, ID, Display Name, Top 50 tracks, Track Audio Features
# print(user_dict['User ID'],
#       user_dict['Display Name'],
#       user_dict['Top Track IDs'],
#       user_dict['Track Audio Features'])


# Track audio features (the dict contains audio feats for all 50 tracks)
print(
    user_dict['Track Audio Features'][0]['danceability'],
    user_dict['Track Audio Features'][0]['energy'],
    user_dict['Track Audio Features'][0]['key'],
    user_dict['Track Audio Features'][0]['loudness'],
    user_dict['Track Audio Features'][0]['mode'],
    user_dict['Track Audio Features'][0]['speechiness'],
    user_dict['Track Audio Features'][0]['acousticness'],
    user_dict['Track Audio Features'][0]['instrumentalness'],
    user_dict['Track Audio Features'][0]['valence'],
    user_dict['Track Audio Features'][0]['liveness'],
    user_dict['Track Audio Features'][0]['tempo'],
    user_dict['Track Audio Features'][0]['duration_ms'],
    user_dict['Track Audio Features'][0]['time_signature']
    )

0.83 0.159 1 -14.461 1 0.0383 0.946 2.02e-05 0.189 0.362 104.95 207400 4


### First we create a user

In [5]:
user1 = User(id=user_dict['User ID'], display_name=user_dict['Display Name'])

In [6]:
# here the user needs to be added to the database so that we can update a `user_id` for the token ForeignKey
user1.id, user1.display_name

('37t3cvb5u3o97hin4bsj40abw', 'Agustinvargas')

In [23]:
# session = Session()
session.add(user1)
session.commit()

### Instantiate token add the user1 data

In [7]:
token1 = Tokens(
        access_token=user_dict['Tokens Info']['access_token'],
        token_type=user_dict['Tokens Info']['token_type'],
        expires_in=user_dict['Tokens Info']['expires_in'],
        refresh_token=user_dict['Tokens Info']['refresh_token'],
        scope=user_dict['Tokens Info']['scope'],
        expires_at=user_dict['Tokens Info']['expires_at'],
        # user_token is the foreign key populated using the user primary key
        user=user1.id
        )

### Instantiate a track

In [7]:
# importing a track into the database for a given user and play list
trk = Tracks(
            id=user_dict['Top Track IDs'][0],
            danceability=user_dict['Track Audio Features'][0]['danceability'],
            energy=user_dict['Track Audio Features'][0]['energy'],
            key=user_dict['Track Audio Features'][0]['key'],
            loudness=user_dict['Track Audio Features'][0]['loudness'],
            mode=user_dict['Track Audio Features'][0]['mode'],
            speechiness=user_dict['Track Audio Features'][0]['speechiness'],
            acousticness=user_dict['Track Audio Features'][0]['acousticness'],
            instrumentalness=user_dict['Track Audio Features'][0]['instrumentalness'],
            valence=user_dict['Track Audio Features'][0]['valence'],
            liveness=user_dict['Track Audio Features'][0]['liveness'],
            tempo=user_dict['Track Audio Features'][0]['tempo'],
            duration_ms=user_dict['Track Audio Features'][0]['duration_ms'],
            time_signature=user_dict['Track Audio Features'][0]['time_signature']
            )

In [10]:

for k, v in enumerate(user_dict['Track Audio Features']):
    globals()['trk' + str(k)] = Tracks(
            id=user_dict['Top Track IDs'][k],
            danceability=v['danceability'],
            energy=v['energy'],
            key=v['key'],
            loudness=v['loudness'],
            mode=v['mode'],
            speechiness=v['speechiness'],
            acousticness=v['acousticness'],
            instrumentalness=v['instrumentalness'],
            valence=v['valence'],
            liveness=v['liveness'],
            tempo=v['tempo'],
            duration_ms=v['duration_ms'],
            time_signature=v['time_signature']
            )
    globals()['play_lst' + str(k)] = UserPlaylist(
            u_id=user1,
            user_ply_lst=globals()['trk' + str(k)]
            )

In [10]:
# If user and track not in DB use:

playlist_test = UserPlaylist(
            u_id=user1.id,
            tracks_id=trk.id
        )

In [11]:
# if user and track in DB use:

playlist_test = UserPlaylist(
            user_id="user in db",
            track_id="track ID in db"
)

<User Playlist(u_id='37t3cvb5u3o97hin4bsj40abw', tracks_id='0akyEssGRVHstqCSWXusJL', uri='None')

In [24]:
# Query to check if user exists in DB, if not returns None

q = session.query(User).filter(User.id == '37t3cvb5u3o97hin4bsj40abw').first()
print(q)

<User(display_name='Agustinvargas')>


In [26]:
print(q is None)

False


In [1]:
import sys
sys.path.insert(0, '../')
from spotify_users import UserData

In [2]:
users_data = UserData()

In [3]:
user1 = users_data.check_for_user()

User Agustinvargas already exists in DataBase;                    checking for token
Agustinvargas has a token, updating it


In [5]:
user_id_2 = '37t3cvb5u3o97hin4bsj40abw'

user2_top_50_aud_feat = users_data.get_playlists_trx(
                                    spot_session=user1['spot_session'],
                                    user2=user_id_2
                                    )

In [7]:
from spotify_users import CreatePlaylist

playlist_gen = CreatePlaylist()
playlist = playlist_gen.create_playlist(
                        user1_top_aud_feat=user1['top_50_aud_feat'],
                        user2_top_aud_feat=user2_top_50_aud_feat,
                        spot_session=user1['spot_session'],
                        user_info=user1['user_info'],
                        user2=user_id_2
                        )

In [8]:
playlist['URI']

'spotify:playlist:3H9kLAxsmyr51cyIqLMON6'